# change to load data from load_data.py


data source: ./simplified_data/simplified_data2.csv

data source: ./simplified_data/simplified_data2.csv

data 處理:

data v2 刪掉所有文字input

deleted 回診,西藥

## train 方法:

.....



In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.utils.class_weight import compute_sample_weight
from tabulate import tabulate

# custom import 
import my_utilities as myutil
import load_data

## Read Data

1. 決定是否要delete 少於某threshold的藥
2. 用load_data裡的`load_data_for_n_med` load data

In [2]:
del_med_under_thres = 0     # 於所有medical cases中出現次數少於此數的藥->整col 刪除
                            # if set to 250 -> will leave 10 medicine
(X_np, X_val_np, train_y, val_y,  num_col_x) = load_data.load_data_for_1_med()

assert(isinstance(X_np, np.ndarray))
assert(isinstance(X_val_np, np.ndarray))
assert(isinstance(train_y, pd.DataFrame))
assert(isinstance(val_y, pd.DataFrame))
assert(isinstance(num_col_x, int))


in ReadData
type of data: <class 'pandas.core.frame.DataFrame'>
Shape of data = (797 rows, 215 cols).
first_medicine:  麻黃
first_medicine_idx:  113
should be 麻黃
SplitXY:
Shape of X = (796 rows, 111 cols). First 10 data of X:
    肺癌  胰臟癌  肝癌  腺癌  攝護腺癌  骨癌  淋巴癌  胃癌  腦瘤  肝炎
1    0    0   0   0     0   0    0   0   0   0
2    0    0   0   0     0   0    0   0   0   0
3    0    0   0   0     0   0    0   0   0   0
4    0    0   0   0     0   0    0   0   0   0
5    0    0   0   0     0   0    0   0   0   0
6    0    0   0   0     0   0    0   0   0   0
7    0    0   0   0     0   0    0   0   0   0
8    0    0   0   0     0   0    0   0   0   0
9    0    0   0   0     0   0    0   0   0   0
10   0    0   0   0     0   0    0   0   0   0
Shape of y = (796 rows, 102 cols). First 10 data of y:
    麻黃  桂枝  荊芥  防風  細辛  白芷  生薑  辛夷  葛根  升麻
1    0   1   0   0   1   1   0   0   0   0
2    0   1   0   0   0   0   1   0   0   0
3    0   0   0   0   0   0   0   0   0   0
4    0   1   0   0   0   0   1  

In [3]:
# data type checking  you can run this if you suspect data type 
# else can skip this cell


# myutil.print_df(val_y, "---- y ----")
# print(val_y)
# checking
# 1. Count occurrence of na, int, float, str in y
na_count = val_y.isna().sum().sum()
str_count = val_y[val_y.map(type) == str].count().sum()
int_count = val_y[val_y.map(type) == int].count().sum()
float_count = val_y[val_y.map(type) == float].count().sum()

print(f"Number of NA values in y: {na_count}")
print(f"Number of str values in y: {str_count}")
print(f"Number of int values in y: {int_count}")
print(f"Number of float values in y: {float_count}")

Number of NA values in y: 0
Number of str values in y: 0
Number of int values in y: 16218
Number of float values in y: 0


## compute class weight

In [4]:

class_weights = compute_sample_weight(class_weight='balanced', y=train_y)
class_weight_dict = dict(enumerate(class_weights))


## Build Model

In [5]:
model = Sequential([
    Dense (units=16, input_shape=(num_col_x,), activation='relu'),
    Dense (units=32, activation='relu'), 
    Dense (units=32, activation='relu'), 
    Dense (units=2, activation='softmax')
])
model.summary()

model.compile(optimizer=Adam(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                1792      
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dense_3 (Dense)             (None, 2)                 66        
                                                                 
Total params: 3,458
Trainable params: 3,458
Non-trainable params: 0
_________________________________________________________________


## Train Model

train medicine-by-medicine

In [6]:
all_res=[]
result_df_dict={}   # dict of df
#loss_acc_dict={}    # dict of loss and accuracy of each medicine
accuracy_dict={}    # dict of accuracy of each medicine

for i in range(train_y.shape[1]):
#for i in range(2):
    res = {}
    res_list =[]
    chosen_col = train_y.iloc[:,i].copy()
    assert(isinstance(chosen_col, pd.Series))
    assert(len(chosen_col) == len(train_y))
    print("processing medicine:", chosen_col.name)
    chosen_y_np = chosen_col.values.astype('float64')

    y_val_chosen_col = val_y.iloc[:,i].copy()
    assert(isinstance(y_val_chosen_col, pd.Series))
    assert(len(y_val_chosen_col) == len(val_y))
    #y_val_chosen_col_np = y_val_chosen_col.values.astype('float64')

    # fit model for this medicine
    model.fit(x=X_np, y=chosen_y_np,
        class_weight=class_weight_dict,  # <---- add class weight
          batch_size=50, epochs=100, shuffle=True, verbose=0)
    
    # make prediction for this medicine
    predictions = model.predict(X_val_np)
    print(type(predictions))
    print("predictions.shape:", predictions.shape)
    
    df_predictions = pd.DataFrame(predictions, columns=["predicted as 0", "predicted as 1"])
    df_predictions["predicted value"] = np.where(df_predictions["predicted as 0"] > df_predictions["predicted as 1"], 0, 1)
    df_predictions["ground truth"] = y_val_chosen_col.values
     # create new column that checks if "predicted value" is equal to "ground truth"
    df_predictions["is_correct"] = df_predictions["predicted value"] == df_predictions["ground truth"]

    accuracy = df_predictions["is_correct"].mean()
    accuracy_dict[chosen_col.name] = accuracy  
    result_df_dict[chosen_col.name] = df_predictions
    

print("training done.")




processing medicine: 麻黃
5/5 [==============================] - 0s 3ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 桂枝
5/5 [==============================] - 0s 5ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 荊芥
5/5 [==============================] - 0s 3ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 防風
5/5 [==============================] - 0s 3ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 細辛
5/5 [==============================] - 0s 5ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 白芷
5/5 [==============================] - 0s 3ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 生薑
5/5 [==============================] - 0s 4ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 辛夷
5/5 [==============================] - 0s 4ms/step
<class 'numpy.ndarray'>
predictions.

KeyboardInterrupt: 

## Handle result

In [ ]:

folder_name = "result/train_1_med_result"
predicted_value_index = None
ground_truth_index = None

# calculate f1 score of for each medicine

for key, df in result_df_dict.items():
    # create 'TP/FP/TN/TN/FN' column

    if predicted_value_index is None:
        predicted_value_index = df.columns.get_loc('predicted value')
    if ground_truth_index is None:
        ground_truth_index = df.columns.get_loc('ground truth')


    df['TP/FP/TN/TN/FN'] = df.apply(lambda row: 'TP' if row[predicted_value_index] == 1 and row[ground_truth_index] == 1 else 
                                 ('FP' if row[predicted_value_index] == 1 and row[ground_truth_index] == 0 else 
                                  ('FN' if row[predicted_value_index] == 0 and row[ground_truth_index] == 1 else 'TN')), axis=1)

# count occurrences of 'TP', 'FP', 'FN', and 'TN'
counts = df['TP/FP/TN/TN/FN'].value_counts()
# get number of 'TP', 'FP', 'FN', and 'TN'
num_tp = counts.get('TP', 0)
num_fp = counts.get('FP', 0)
num_fn = counts.get('FN', 0)
num_tn = counts.get('TN', 0)

f1_score = 2 * num_tp / (2 * num_tp + num_fp + num_fn)
    
for key, df in result_df_dict.items():
    print(f"DataFrame for {key}:")
    myutil.print_df(df)
    myutil.df_to_csv(df, save_path="./result/top10med_result", file_prefix=key)
   


